<a href="https://colab.research.google.com/github/eddielin0926/kaggle/blob/main/notebooks/digit-recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Digit Recognizer](https://www.kaggle.com/competitions/digit-recognizer)
Learn computer vision fundamentals with the famous MNIST data


## Preparation

In [1]:
! pip install --upgrade pip
! pip install --upgrade kaggle flaml
! pip install git+https://github.com/eddielin0926/myutils.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 10.0 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.6-py3-none-any.whl size=111943 sha256=98f5c7b6281257dab6746d77b9e4fd839eb5e37e44b7c6ad35ff084e244cdc08
  Stored in directory: /root/.cache/pip/wheels/53/34/8c/8ca3450d17206d9e37e1ee3aeb47cbb2873d22a9e0c60eb137
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16
  Cloning https://github.com/eddielin0926/myutils.git to /tmp/pip-req-build-jedo84d0
  Running command git clone --filter=blob:none --qui

Set environment variable for kaggle authentication from Colab Secrets.

In [2]:
import os
from google.colab import userdata

os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

Set competition name.

In [3]:
COMPETITION = "digit-recognizer"

Download competition dataset.

In [4]:
! mkdir -p data
! kaggle competitions download -c {COMPETITION}
! unzip -o {COMPETITION}.zip -d data

 65% 10.0M/15.3M [00:00<00:00, 39.3MB/s]
100% 15.3M/15.3M [00:00<00:00, 54.8MB/s]
Archive:  digit-recognizer.zip
  inflating: data/sample_submission.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          


Import libraries.

In [5]:
import pandas as pd
import numpy as np

import myutils

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore")

### Datasets

Read training and testing dataset.

In [6]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

### Parameters

In [7]:
ID = "ImageId"
TARGET = "Label"

## Exploratory Data Analysis

### Train Dataset

In [8]:
display(df_train.shape)
display(myutils.overview(df_train))

(42000, 785)

,Column,Dtype,Null,Non-Null,Count,Unique,Mode,Mean,Std,Overview,Sample
0,label,int64,0,42000,42000,10,1,4.456643,2.887730,"[0.0, 2.0, 4.0, 7.0, 9.0]",1
1,pixel0,int64,0,42000,42000,1,0,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
2,pixel1,int64,0,42000,42000,1,0,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
3,pixel2,int64,0,42000,42000,1,0,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
4,pixel3,int64,0,42000,42000,1,0,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
...,...,...,...,...,...,...,...,...,...,...,...
780,pixel779,int64,0,42000,42000,3,0,0.002857,0.414264,"[0.0, 0.0, 0.0, 0.0, 62.0]",0
781,pixel780,int64,0,42000,42000,1,0,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
782,pixel781,int64,0,42000,42000,1,0,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
783,pixel782,int64,0,42000,42000,1,0,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0


### Test Dataset

In [9]:
display(df_test.shape)
display(myutils.overview(df_test))

(28000, 784)

,Column,Dtype,Null,Non-Null,Count,Unique,Mode,Mean,Std,Overview,Sample
0,pixel0,int64,0,28000,28000,1,0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
1,pixel1,int64,0,28000,28000,1,0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
2,pixel2,int64,0,28000,28000,1,0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
3,pixel3,int64,0,28000,28000,1,0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
4,pixel4,int64,0,28000,28000,1,0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
...,...,...,...,...,...,...,...,...,...,...,...
779,pixel779,int64,0,28000,28000,1,0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
780,pixel780,int64,0,28000,28000,1,0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
781,pixel781,int64,0,28000,28000,1,0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
782,pixel782,int64,0,28000,28000,1,0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0


## Data Preprocessing

### Encoding

In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
encode_cols = []
for col in encode_cols:
    df_train[col] = le.fit_transform(df_train[col])
    df_test[col] = le.transform(df_test[col])
    print('Encoding:', col, le.classes_)

In [ ]:
train = pd.get_dummies(df_train.drop([], axis=1))
test = pd.get_dummies(df_test.drop([], axis=1))

In [ ]:
drop_cols = []
for col, corr in zip(train.corr()[TARGET].index, train.corr()[TARGET]):
    if abs(corr) < 0.2:
        drop_cols.append(col)
print('Dropping cols:', drop_cols)
train = train.drop(drop_cols, axis=1)
test = test.drop(list(set(drop_cols) & set(test.columns)), axis=1)

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(train.corr(), annot=False, annot_kws={'size': 8}, vmin=-1, vmax=1, cmap='viridis')

## Training

In [ ]:
from flaml import AutoML
from sklearn.ensemble import GradientBoostingRegressor

automl = AutoML()
automl_settings = {
    "task": "classification",
    "time_budget": 120,
    "log_file_name": f"{COMPETITION}.log",
    "estimator_list": ['lgbm', 'xgboost', 'xgb_limitdepth', 'rf', 'extra_tree'],
    "metric": "ap",
    "ensemble": {
        "final_estimator": GradientBoostingRegressor(),
        "passthrough": True,
    },
}
automl.fit(dataframe=train, label="Target", **automl_settings)

In [ ]:
from pprint import pprint

print('Best ML leaner:', automl.best_estimator)
print("Best score:", automl.score(train.drop([TARGET], axis=1), train[TARGET]))
print('Best hyperparmeter config:')
pprint(automl.best_config)

In [ ]:
from flaml.automl.data import get_output_from_log

time_history, best_valid_loss_history, valid_loss_history, config_history, metric_history = \
    get_output_from_log(filename=automl_settings["log_file_name"], time_budget=120)

plt.title("Learning Curve")
plt.xlabel("Wall Clock Time (s)")
plt.ylabel("Validation Accuracy")
plt.step(time_history, 1 - np.array(best_valid_loss_history), where="post")
plt.show()

## Submission

In [ ]:
predicted = automl.predict(test)
submission = pd.DataFrame({ID: df_test[ID], TARGET: predicted})
submission.to_csv('./submission.csv', index=False)
submission.head()

In [ ]:
! kaggle competitions submit -c {COMPETITION} -f submission.csv -m "First try"

In [ ]:
! kaggle competitions submissions {COMPETITION}

## Reference